https://python.langchain.com/docs/get_started/quickstart/

In [ ]:
#!pip install langchain
#!pip install langchain-openai

In [1]:
from dotenv import load_dotenv
load_dotenv()
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="..."
# export OPENAI_API_KEY="..."

True

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

# Prompt engineering 

In [4]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing by providing automated testing tools, testing frameworks, and test case generation capabilities. It can also facilitate the creation of test scripts and help in running tests on different platforms and environments. Additionally, Langsmith can assist in analyzing test results, identifying bugs and issues, and providing insights for improving test coverage and efficiency. Overall, Langsmith can streamline the testing process and enhance the overall quality of the software being tested.', response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 15, 'total_tokens': 102}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-71ac9643-025d-4967-8322-3c6da09190af-0')

In [8]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])
chain = prompt | llm 
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content="Langsmith can help with testing in various ways:\n\n1. Test Automation: Langsmith can be used to automate testing scenarios, allowing for more efficient and consistent testing of software applications. By writing test scripts using Langsmith, testers can easily simulate user interactions and verify the functionality of the application.\n\n2. Data Generation: Langsmith can be used to generate realistic test data for testing purposes. This can help in creating diverse test cases to validate the application's behavior under different scenarios.\n\n3. Integration Testing: Langsmith can be utilized for integration testing by simulating interactions between different components of the software system. This can help in identifying any integration issues and ensuring that the components work seamlessly together.\n\n4. Load Testing: Langsmith can be employed for load testing to simulate a large number of concurrent users accessing the application. This can help in assessing t

In [9]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt | llm | output_parser
chain.invoke({"input": "how can langsmith help with testing?"})

"Langsmith can help with testing in various ways, including:\n\n1. Automated Testing: Langsmith can be integrated with testing frameworks such as JUnit, TestNG, or Selenium to automate the testing process. This can help in running tests quickly and efficiently, reducing the time and effort required for manual testing.\n\n2. Test Data Generation: Langsmith can be used to generate test data for different scenarios, helping in creating a comprehensive test suite. This can ensure better test coverage and identify potential issues early in the development cycle.\n\n3. Performance Testing: Langsmith can also be used for performance testing by simulating a large number of users and analyzing the system's response under various load conditions. This can help in identifying performance bottlenecks and optimizing the system for better scalability.\n\n4. Integration Testing: Langsmith can assist in integration testing by facilitating the testing of different components or modules of an applicatio

# Retrieval Chain

In [14]:
#!pip install beautifulsoup4
#!pip install faiss-cpu

In [11]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [12]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [15]:
# local vectorstore, FAISS https://python.langchain.com/docs/integrations/vectorstores/faiss/
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [18]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])


LangSmith can help with testing by allowing developers to create datasets, run tests on LLM applications using these datasets, and easily compare results for different configurations side-by-side. Additionally, LangSmith provides a playground environment for rapid iteration and experimentation, as well as features for beta testing, capturing feedback, annotating traces, adding runs to datasets, and monitoring performance metrics over time.
